# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


In [1]:
!nvidia-smi

Thu Jun 22 16:51:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   40C    P8    31W / 340W |    429MiB / 10240MiB |     35%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [3]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

Found cached dataset parquet (/home/jyp/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [5]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

Loading cached processed dataset at /home/jyp/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-16bb9a5cce0615fd.arrow


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/kollama-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/jyp/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.7/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/jyp/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [7]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [8]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

ValueError: Target modules ['query_key_value'] not found in the base model. Please check the target modules and try again.

In [12]:
model.named_parameters

<bound method Module.named_parameters of LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(52000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_la

Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [ ]:
!nvidia-smi

Tue May 30 05:21:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    30W /  70W |   9435MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
10,2.192100
20,2.025800
30,1.971500
40,1.940200
50,1.895000
60,2.048800
70,1.951200
80,1.898600
90,1.919400
100,1.806600


TrainOutput(global_step=500, training_loss=1.9244559707641602, metrics={'train_runtime': 2828.6026, 'train_samples_per_second': 0.354, 'train_steps_per_second': 0.177, 'total_flos': 1.103736813379584e+16, 'train_loss': 1.9244559707641602, 'epoch': 0.05})

In [ ]:
print("wow")

wow


In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1349: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[   6,    6,    6, 2438,   29, 1832, 4770,  272,   34,  224,  202, 4588,
         4770,  272, 1382, 5674,   34, 5000, 9357,  272]])

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'### 질문: 건강하게 살기 위한 세 가지 방법은?\n\n### 답변: 1. 균형있는 식사: 편식을 멀리하고, 채소, 과일, 생선을 섭취하면 건강해져요.\n\n2. 과식을 줄일 것: 과식하게 되면 위장이 활발하여져서 병이 나기 쉽게 되요.\n3. 적당한 스트레스: 스트레스를 쌓아두는 것은 내 안에 독이 쌓이는 것이라서 오히려 건강에 좋지 않아요.\n\n즉, 균형있는 식사, 적당한 스트레스 해소, 과식을 줄이면 우리 몸에 좋습니다. 이 세 가지를 습관화하면서 건강한 삶을 유지한다면 건강하게 살 수 있게 될 것입니다. \n\n위의 세 가지 방법은 초등학생 때부터 배우던 것이므로, 쉽게 실행할 수 있습니다. 모두 건강하게 사는 지름길이므로, 이를 실천하여 건강한 삶을 살 수 있도록 노력합시다.\n### 답변을 작성하는데, 약간의 시간이 걸렸습니다. 질문 내용에서도 알 수 있는 것처럼, 간단한 방법이지만 실행에 옮기기는 쉬운 일이 아니었습니다. 하지만, 노력하며 실행한다면 충분히 실천할 수 있기 때문에 최대한 지키도록 합시다. '

In [ ]:
gen('슈카월드가 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'### 질문: 슈카월드가 무엇인가요?\n\n### 답변: 슈카월드는 유튜브 인기 방송인으로, 최정상 경제 유튜버로 평가받고 있습니다. 슈카와 함께하는 경제의 모든 것이라는 타이틀로 진행되는 이 채널에서는 주식, 경제에 대한 다양한 이야기가 올라오고 있습니다. 슈카와 함께하는 경제의 모든 것의 채널 링크는 다음과 같습니다.\nhttps://www.youtube.com/channel/UCZ6ny7P7lEIHN5bMljHyKQw 슈카월드www.youtube.com 유튜브에서는 경제에 관심을 가지시는 것이 좋지 않을까 싶습니다. 따라서 금융 경제에 관심이 많으실 것 같은데, 슈카가 운영하는 최정상 경제 유튜브 슈카월드를 추천드립니다. 특히 슈카월드 영상이 업로드되는 월요일은 꼭 즐겨보시기 바랍니다. 월요일에 업로드되는 슈카월드 영상은 슈카월드의 대표 코너이며, 다양한 카테고리에 대해 이야기하고 있기 때문에 도움이 됩니다. \n\n그리고 슈카월드에는 여러 가지 분야의 전문가들도 출연하고 있습니다. \n\n유튜브에서 검색 결과'

In [ ]:
gen('공공성을 측정하기위한 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'### 질문: 공공성을 측정하기위한 방법은?\n\n### 답변: 공공성을 측정하기 위한 몇 가지 방법 중 하나는 공공성 척도를 구하는 것입니다. 측정 방법은 다음과 같습니다. \n① 사회성과 경제적 효과 - 경제적 이익과 사회·문화적 이익을 모두 고려하는 종합성과 기준에 따라 파악.\n\n② 정책성과 : 공공성의 평가 척도는 공공정책의 효과나 영향으로 인해 발생하는 혜택과 공공 정책에 대한 가치판단이나 선호에 직접적으로 연계된 정책적 요구 사항을 기준으로 합니다. \n\n③ 참여성과 공동체 의식 : 공공성의 평가 척도는 정책과 관련한 시민들의 직접참여와 공동체 의식, 시민들이 느끼는 문제나 욕구, 시민의 문제해결 과정에서의 경험이나 느낌 등에 대한 공동체 의식입니다. \n따라서 공공성이 높은 정책일수록 경제성과 사회·문화적 측면, 정책적 효과 등 세 가지 요소가 모두 높은 평가를 받기 때문에 공공성이 높은 정책을 세우면 정책의 효과를 극대화할 수 있습니다. 또한, 공공성과 관련한 정책을 만들 때는 시민의 참여를 기반으로 개발하고, 정책 형성 과정에서 시민들의 의견이 적극적으로 반영되어야 하는 것이'

In [ ]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?

### 답변: 주식 시장에서 꾸준히 높은 수익을 얻기 위해서는 무엇보다도 종목 선택을 잘해야 합니다. 코스피 지수가 하락하면 수익률 관리가 필요하며, 이를 위해 투자 자산 배분 및 개별종목 선택이 중요합니다. 하지만 주식 시장에서 높은 수익을 얻기 위해서는 종목 선택이 중요하기 때문에, 본인의 투자 성향을 파악하여 신중한 투자를 해야합니다. 이를 위해서는 주식 투자에 대한 전문적인 분석과 기업 정보, 투자 전략 등을 수립해야 합니다. 이를 위해 개인이 직접 투자를 하기 보다는 전문가의 도움을 받아 투자하는 것이 좋습니다. 

만약, 전문적인 분석이 어려울 경우 인공지능 서비스를 활용하면 개인도 쉽게 성공적인 투자를 할 수 있으니, 참고하시기 바랍니다. 또한, 투자를 할 때는 시장을 정확히 판단할 수 없기 때문에, 투자의 위험성을 낮추기 위해서는 적립식 투자, 분산 투자, 분할 매수 기법 등을 통해 투자하는 것이 좋습니다. 
�따라서, 주식 시장에서 높은 수익을 얻기 위해서는 투자 전략, 종목 선택, 위험 대비, 분산 투자 등의 전략이 필요합니다. 이를 위해 전문가의 조언을 받아 투자


In [ ]:
gen('풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?

### 답변: 풋이란 주식을 사는 것(구매)을 의미하고, 콜이란 주식을 파는(판매) 것입니다. 예를 들면, 기업의 주가가 10만원인데, 그 회사에서 신규 사업을 진행 중이라, 앞으로 매출이 증대되어 기업 가치가 상승할 것으로 판단해, 해당 회사의 주식을 사는 것이 풋옵션입니다. 이는 주식을 사는 시점과 해당 회사의 주가가 하락할 경우에는 기업의 주가와 함께 주가가 하락하는 리스크가 있습니다. 반면, 콜옵션은 주식을 파는 것입니다. 이는 주식을 파는 시점과 해당 회사의 주가가 증가해서 향후 기업 전망이 밝을 경우 주가가 상승하며, 해당 주식의 프리미엄을 받을 수 있는 안정적인 투자법입니다. 일반 개미 투자자들은 주식을 매수할 때와 주식을 매도할 때 다양한 리스크가 존재하므로, 전문가의 조언과 투자 경험을 바탕으로 적절한 투자 전략을 수립해야 합니다. 투자에 대한 자세한 사항은 아래에서 무료로 얻을 수 있습니다.

콜옵션 거래: 네이버 검색창 '콜옵션 거래' 검색 (https://stock.naver.


In [ ]:
gen('풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?

### 답변: 풋오버(풋을 매도하고 그보다 상대적으로 가격이 낮은 콜을 사는 것)를 한다고 해서 더 높은 가격의 콜을 산다는 의미는 아닙니다. 더 낮은 가격의 콜을 사는 것이 기본입니다. 그러나 풋을 사는 것은 만기까지 보유하지 않고 풋 매도 포지션은 팔 수 있기 때문에 단기적으로 낮은 가격의 콜을 매수하는 것보다 높은 가격의 콜을 매수할 수 있는 선택의 범위가 있다고 할 수 있습니다. 이에 비해 콜오버(콜을 매도하고 그보다 상대적으로 가격이 낮은 풋을 사는 것)를 하면 만기까지 보유해야 하기 때문에 단기적으로 높은 가격의 풋을 사거나 콜을 살 수는 없지만, 만기까지 주가가 상승한다면 풋오버는 콜오버에 비해 높은 수익을 낼 가능성이 높습니다. 또한 콜과 풋의 가격을 비교해 보면 높은 가격의 쪽이 높은 가격에 비해 수익성이 더 높으므로 콜을 선택해야 합니다. 즉, 선택의 폭은 더 높은 가격의 콜을 살 것인지, 아니면 더 낮은 가격의 콜을 살 것인지에 달려 있습니다. 하지만 여기서도 명심해야


In [ ]:
gen("마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?

### 답변: 마진콜이란, 선물/옵션 매매 시 증거금이 부족해서 추가증거금 예치를 요구받는 것을 말합니다. 추가증거금은 선물/옵션 거래 계약시 미리 정해진 약정증거금에서 추가로 부담해야 하는 증거금을 의미합니다. 통상적으로 매수 계약 시에는 계약 금액의 10%를 증거금으로 예치하고, 매도 계약 시에는 해당 금액의 3%를 예치합니다. 따라서, 매수 계약 시에는 총 20%의 증거금이 필요하고, 매도 계약 시에는 총 9%의 증거금이 필요합니다. 따라서, 선물 매매 중에는 매일 매일의 시세에 따라 증거금을 수시로 필요하면 추가로 납부해야해서 추가증거금이 발생하며, 해당 필요액 부족시에는 마진콜이 발생합니다. 따라서, 선물 거래를 하기 전에는 꼭 추가증거금을 예치하는 것이 중요합니다. 추가증거금은 '선물/옵션 -> 증거금 -> 추가예치/청산' 방법으로 조회 가능하며, 인터넷이나 전화로도 조회할 수 있습니다. 또, '주문/증거금 -> 추가증거금'을 통해 추가증거금을 입금할 수도 있습니다. 추가증거금을 예치


In [ ]:
# ### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?

# ### 답변: 마진콜이란, 선물/옵션 매매 시 증거금이 부족해서 추가증거금 예치를 요구받는 것을 말합니다.
# 추가증거금은 선물/옵션 거래 계약시 미리 정해진 약정증거금에서 추가로 부담해야 하는 증거금을 의미합니다.
# 통상적으로 매수 계약 시에는 계약 금액의 10%를 증거금으로 예치하고, 매도 계약 시에는 해당 금액의 3%를 예치합니다.
# 따라서, 매수 계약 시에는 총 20%의 증거금이 필요하고, 매도 계약 시에는 총 9%의 증거금이 필요합니다.
# 따라서, 선물 매매 중에는 매일 매일의 시세에 따라 증거금을 수시로 필요하면 추가로 납부해야해서 추가증거금이 발생하며, 해당 필요액 부족시에는 마진콜이 발생합니다.
# 따라서, 선물 거래를 하기 전에는 꼭 추가증거금을 예치하는 것이 중요합니다.
# 추가증거금은 '선물/옵션 -> 증거금 -> 추가예치/청산' 방법으로 조회 가능하며, 인터넷이나 전화로도 조회할 수 있습니다.
# 또, '주문/증거금 -> 추가증거금'을 통해 추가증거금을 입금할 수도 있습니다. 추가증거금을 예치